In [ ]:
!pip install librosa

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
import librosa
import librosa.display
import IPython.display as ipd

In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm

metadata = pd.read_csv("/content/drive/MyDrive/Minor Project/UrbanSound8K/metadata/UrbanSound8K.csv")
print(metadata)

         slice_file_name    fsID       start         end  salience  fold  \
0       100032-3-0-0.wav  100032    0.000000    0.317551         1     5   
1     100263-2-0-117.wav  100263   58.500000   62.500000         1     5   
2     100263-2-0-121.wav  100263   60.500000   64.500000         1     5   
3     100263-2-0-126.wav  100263   63.000000   67.000000         1     5   
4     100263-2-0-137.wav  100263   68.500000   72.500000         1     5   
...                  ...     ...         ...         ...       ...   ...   
8727     99812-1-2-0.wav   99812  159.522205  163.522205         2     7   
8728     99812-1-3-0.wav   99812  181.142431  183.284976         2     7   
8729     99812-1-4-0.wav   99812  242.691902  246.197885         2     7   
8730     99812-1-5-0.wav   99812  253.209850  255.741948         2     7   
8731     99812-1-6-0.wav   99812  332.289233  334.821332         2     7   

      classID             class  
0           3          dog_bark  
1           2  chil

In [10]:

def features_extractor(file):
  data,sr = librosa.load(filename , res_type='kaiser_fast')
  mfccs_features = librosa.feature.mfcc(y = data , sr=sr , n_mfcc=40)
  mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
  return mfccs_scaled_features



In [ ]:
import numpy as np 
import os as os
from tqdm import tqdm
extracted_features = []
for index_num,row in tqdm(metadata.iterrows()):
  filename = os.path.join(os.path.abspath("/content/drive/MyDrive/Minor Project/UrbanSound8K/audio/"),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
  final_class_labels= row["class"]
  data=features_extractor(filename)
  extracted_features.append([data,final_class_labels])



In [ ]:
Data = pd.DataFrame(extracted_features,columns=["Features","Class"])
Features = np.array(Data["Features"].tolist())

Class = np.array(Data["Class"].tolist())
print(Features)

print(Class)

In [ ]:
Class_ = pd.get_dummies(Class)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Features,Class_,train_size=0.2,random_state=0)

## Neural Architecture

In [3]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
!pip install keras-tuner

In [4]:
Model =tf.keras.models.load_model("/content/drive/MyDrive/Minor Project/Saved Model")

In [ ]:
model = Sequential()
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
Model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               4100      
                                                                 
 activation_8 (Activation)   (None, 100)               0         
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 200)               20200     
                                                                 
 activation_9 (Activation)   (None, 200)               0         
                                                                 
 dropout_7 (Dropout)         (None, 200)               0         
                                                                 
 dense_10 (Dense)            (None, 100)              

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 110
num_batch_size = 16

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.7409104108810425


In [ ]:
model.save("/content/drive/MyDrive/Minor Project/Saved Model/")

Model Saved using joblib

In [5]:
from joblib import dump
dump(Model,'Model_jb.joblib')

INFO:tensorflow:Assets written to: ram://a1e84544-610f-4789-b4ed-42226c9cc06e/assets


['Model_jb.joblib']

In [7]:
from joblib import load
model_joblib = load('Model_jb.joblib') 

Model Saved Using Pickle

In [ ]:
import pickle
pickle.dump(Model,open("model.pkl","wb"))

INFO:tensorflow:Assets written to: ram://9bdf0752-46de-4aef-bd2e-6b66e99ff581/assets


In [ ]:
model.save("C:\Users\Anonymous\Documents")

In [ ]:
import pickle
model_pickel = pickle.load(open('/content/model.pkl','rb'))

## Prediction 

In [18]:
classes = {1:"Car horn",2:"Children Playing",3:"Dog Bark",4 : "drilling",5 : "engine_idling",6 : "gun_shot",7 :"jackhammer",8:"siren",9 : "street_music"}
def features_extractor(file):
  data,sr = librosa.load(filename , res_type='kaiser_fast')
  mfccs_features = librosa.feature.mfcc(y = data , sr=sr , n_mfcc=40)
  mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
  return mfccs_scaled_features

def Predict_classes(filename):
  classes = {1:"Car horn",2:"Children Playing",3:"Dog Bark",4 : "drilling",5 : "engine_idling",6 : "gun_shot",7 :"jackhammer",8:"siren",9 : "street_music"}
  prediction_feature=features_extractor(filename)
  prediction_feature =prediction_feature.reshape(1,-1)
  prediction = model_joblib.predict(prediction_feature)
  class_ =np.where(prediction == np.max(prediction))[1]
  return classes[int(class_)]


In [19]:
filename="/content/drive/MyDrive/Minor Project/UrbanSound8K/audio/fold7/99812-1-6-0.wav"
Predict_classes(filename)

'Car horn'